<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Re_CountSlopes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')

In [3]:
lab.isnull().any()

LabNo              False
PatientUID         False
PatientVisitUID    False
Location            True
RequestItemName    False
ResultItemName      True
CWhen              False
ResultDttm          True
ResultValue         True
Unit                True
ReferenceRange      True
dtype: bool

##**🗂️ Data eGFR of all Patient**

In [4]:
# Define the values to match
result_item_names = ['eGFR(CKD-EPI) age >=18']

# Select rows where 'ResultItemName' matches the specified values
eGFR = lab[lab['ResultItemName'].isin(result_item_names)]
eGFR
labeGFR = eGFR[['LabNo','PatientUID', 'PatientVisitUID', 'ResultItemName', 'ResultValue', 'CWhen']]
labeGFR

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
248,1,2299,3386255,eGFR(CKD-EPI) age >=18,71.7,2559-08-15 09:51:54.913
255,1,2299,3640924,eGFR(CKD-EPI) age >=18,66.8,2559-11-11 09:40:06.053
271,1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2560-03-31 10:18:54.363
286,1,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2560-07-05 10:40:40.763
331,1,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2560-10-03 09:19:54.710
...,...,...,...,...,...,...
21628049,159,925649,3617725,eGFR(CKD-EPI) age >=18,77.60,2560-04-17 08:54:35.740
21628297,160,925649,3617725,eGFR(CKD-EPI) age >=18,72.00,2560-04-19 08:41:45.060
21628538,161,925649,3617725,eGFR(CKD-EPI) age >=18,62.90,2560-04-21 08:47:03.167
21628775,162,925649,3617725,eGFR(CKD-EPI) age >=18,59.60,2560-04-24 08:59:53.310


In [5]:
labeGFR.isnull().any()

LabNo              False
PatientUID         False
PatientVisitUID    False
ResultItemName     False
ResultValue         True
CWhen              False
dtype: bool

In [6]:
labeGFR[labeGFR['ResultValue'].isnull()]

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
15974112,2,2119384,7663675,eGFR(CKD-EPI) age >=18,NaN,2563-09-16 10:36:08.937


In [7]:
labeGFR.count()

LabNo              518711
PatientUID         518711
PatientVisitUID    518711
ResultItemName     518711
ResultValue        518710
CWhen              518711
dtype: int64

## **🗒 Pivot each patientID**
to look at the value they have

In [8]:
import pandas as pd

# Assuming you have a DataFrame named 'eGFR' with the specified columns
pivot_table = pd.pivot_table(labeGFR, index=['PatientUID', 'PatientVisitUID'], values=['ResultItemName', 'ResultValue', 'CWhen'], aggfunc='first')
pivot_table

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue  
PatientUID PatientVisitUID              
2299       3386255                71.7  
           3640924                66.8  
           4020970               67.20  
           4286612               69.20  
           4546471               63.60  
...                                ...  
3432056    10312896              72.86  
           10408954              68.69  
           10573233              45.08  
3437526    10191122              55.94  
           10541118              56.99  

[355985 rows x 3 columns]

**after pivot missing value is disappeared**

In [9]:
pivot_table.isnull().any()

CWhen             False
ResultItemName    False
ResultValue       False
dtype: bool

##  **🕑 generate datetime**
    Split CWhen เพราะต้องการ generate year as คศ. Pandas มี outbound
    ให้อยู่ในระยะเวลาของ คศ. แต่ลอง convert column 'CWhen'
    เป็น datetime แล้วทำไม่ได้เลยต้อง split แล้วค่อย merge

In [10]:
import pandas as pd

df = pd.DataFrame(pivot_table)

# Splitting the 'time' column into separate columns
df[['date', 'timestamp']] = df['CWhen'].str.split(' ', expand=True)
df[['year', 'month', 'day']] = df['date'].str.split('-', expand=True)
df[['hour', 'minute', 'second']] = df['timestamp'].str.split(':', expand=True)
df[['second', 'nanosecond']] = df['second'].str.split('.', expand=True)

# Convert year column from BE to AD
df['year'] = df['year'].astype(int) - 543

# Combine columns into a datetime column
df['date'] = pd.to_datetime(df[['year', 'month', 'day',]])
# Convert timestamp to Timedelta
df['timestamp'] = pd.to_timedelta(df['timestamp'])

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Merge timestamp and date columns
df['datetime'] = df['date'] + df['timestamp']

In [11]:
df_sorted = df.sort_values(by=['PatientUID', 'datetime'])

In [12]:
df_sorted

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2299       3386255          2559-08-15 09:51:54.913  eGFR(CKD-EPI) age >=18   
           3640924          2559-11-11 09:40:06.053  eGFR(CKD-EPI) age >=18   
           4020970          2560-03-31 10:18:54.363  eGFR(CKD-EPI) age >=18   
           4286612          2560-07-05 10:40:40.763  eGFR(CKD-EPI) age >=18   
           4546471          2560-10-03 09:19:54.710  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3432056    10312896         2566-01-30 16:16:45.013  eGFR(CKD-EPI) age >=18   
           10408954         2566-02-24 21:05:12.747  eGFR(CKD-EPI) age >=18   
           10573233         2566-04-10 09:51:32.587  eGFR(CKD-EPI) age >=18   
3437526    10191122         2565-12-26 09:29:11.307  eGFR(CKD-EPI) age >=18   
           10541118         2566-03-30 18:44:40.160  eGFR(CKD-EPI) age >=18   

                           ResultValue       date              timestamp  \
PatientUID PatientVisitUID                                                 
2299       3386255                71.7 2016-08-15 0 days 09:51:54.913000   
           3640924                66.8 2016-11-11 0 days 09:40:06.053000   
           4020970               67.20 2017-03-31 0 days 10:18:54.363000   
           4286612               69.20 2017-07-05 0 days 10:40:40.763000   
           4546471               63.60 2017-10-03 0 days 09:19:54.710000   
...                                ...        ...                    ...   
3432056    10312896              72.86 2023-01-30 0 days 16:16:45.013000   
           10408954              68.69 2023-02-24 0 days 21:05:12.747000   
           10573233              45.08 2023-04-10 0 days 09:51:32.587000   
3437526    10191122              55.94 2022-12-26 0 days 09:29:11.307000   
           10541118              56.99 2023-03-30 0 days 18:44:40.160000   

                            year month day hour minute second nanosecond  \
PatientUID PatientVisitUID                                                 
2299       3386255          2016    08  15   09     51     54        913   
           3640924          2016    11  11   09     40     06        053   
           4020970          2017    03  31   10     18     54        363   
           4286612          2017    07  05   10     40     40        763   
           4546471          2017    10  03   09     19     54        710   
...                          ...   ...  ..  ...    ...    ...        ...   
3432056    10312896         2023    01  30   16     16     45        013   
           10408954         2023    02  24   21     05     12        747   
           10573233         2023    04  10   09     51     32        587   
3437526    10191122         2022    12  26   09     29     11        307   
           10541118         2023    03  30   18     44     40        160   

                                          datetime  
PatientUID PatientVisitUID                          
2299       3386255         2016-08-15 09:51:54.913  
           3640924         2016-11-11 09:40:06.053  
           4020970         2017-03-31 10:18:54.363  
           4286612         2017-07-05 10:40:40.763  
           4546471         2017-10-03 09:19:54.710  
...                                            ...  
3432056    10312896        2023-01-30 16:16:45.013  
           10408954        2023-02-24 21:05:12.747  
           10573233        2023-04-10 09:51:32.587  
3437526    10191122        2022-12-26 09:29:11.307  
           10541118        2023-03-30 18:44:40.160  

[355985 rows x 13 columns]

## 📈 calculate slope by **ResultValue** and **datetime**



> **Calculate slope**
```
 ให้ทำการคำนวณสโลปเฉพาะค่าที่ time_diff ห่างกันอย่างน้อย 7 วันเป็นอย่างต่ำ ถ้าน้อยกว่านั้นไม่นำมาคำนวณหาสโลป
```



In [13]:
import pandas as pd

# Assuming you have a DataFrame 'df_sorted' with columns 'eGFR', 'datetime', 'PatientUID', and 'PatientVisitUID'

# Convert 'ResultValue' column to numeric
df_sorted['eGFR'] = pd.to_numeric(df_sorted['ResultValue'], errors='coerce')

# Convert 'datetime' column to datetime type
df_sorted['datetime'] = pd.to_datetime(df_sorted['datetime'])

# Drop rows with missing values in 'eGFR' and 'datetime' columns
df_sorted.dropna(subset=['eGFR', 'datetime'], inplace=True)

# Sort DataFrame by 'PatientUID' and 'datetime'
df_sorted.sort_values(by=['PatientUID', 'datetime'], inplace=True)

##### Calculate the time difference in days ####
df_sorted['time_difference'] = (df_sorted['datetime'] - df_sorted['datetime'].shift()).dt.days

# Drop rows where time difference is not greater than or equal to 7 days
df_sorted = df_sorted[df_sorted['time_difference'] >= 7]

# Calculate slopes
df_sorted['slopes'] = (df_sorted['eGFR'].diff() / df_sorted['time_difference'])

# Select desired columns
eGFR_Patient = df_sorted[['ResultItemName', 'eGFR', 'datetime', 'slopes', 'time_difference']]

<ipython-input-13-c7b9e7c1a664>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sorted['slopes'] = (df_sorted['eGFR'].diff() / df_sorted['time_difference'])


In [14]:
eGFR_Patient

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2299       3640924          eGFR(CKD-EPI) age >=18  66.80   
           4020970          eGFR(CKD-EPI) age >=18  67.20   
           4286612          eGFR(CKD-EPI) age >=18  69.20   
           4546471          eGFR(CKD-EPI) age >=18  63.60   
           4801938          eGFR(CKD-EPI) age >=18  58.40   
...                                            ...    ...   
3432056    10235244         eGFR(CKD-EPI) age >=18  48.51   
           10306813         eGFR(CKD-EPI) age >=18  67.71   
           10408954         eGFR(CKD-EPI) age >=18  68.69   
           10573233         eGFR(CKD-EPI) age >=18  45.08   
3437526    10541118         eGFR(CKD-EPI) age >=18  56.99   

                                          datetime    slopes  time_difference  
PatientUID PatientVisitUID                                                     
2299       3640924         2016-11-11 09:40:06.053       NaN             87.0  
           4020970         2017-03-31 10:18:54.363  0.002857            140.0  
           4286612         2017-07-05 10:40:40.763  0.020833             96.0  
           4546471         2017-10-03 09:19:54.710 -0.062921             89.0  
           4801938         2018-01-04 09:03:28.847 -0.056522             92.0  
...                                            ...       ...              ...  
3432056    10235244        2023-01-09 12:31:53.220  0.168400             25.0  
           10306813        2023-01-27 12:22:43.000  1.129412             17.0  
           10408954        2023-02-24 21:05:12.747  0.039200             25.0  
           10573233        2023-04-10 09:51:32.587 -0.536591             44.0  
3437526    10541118        2023-03-30 18:44:40.160  0.126702             94.0  

[316328 rows x 5 columns]

In [15]:
t = eGFR_Patient.reset_index()
t

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
0,2299,3640924,eGFR(CKD-EPI) age >=18,66.80,2016-11-11 09:40:06.053,NaN,87.0
1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2017-03-31 10:18:54.363,0.002857,140.0
2,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2017-07-05 10:40:40.763,0.020833,96.0
3,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2017-10-03 09:19:54.710,-0.062921,89.0
4,2299,4801938,eGFR(CKD-EPI) age >=18,58.40,2018-01-04 09:03:28.847,-0.056522,92.0
...,...,...,...,...,...,...,...
316323,3432056,10235244,eGFR(CKD-EPI) age >=18,48.51,2023-01-09 12:31:53.220,0.168400,25.0
316324,3432056,10306813,eGFR(CKD-EPI) age >=18,67.71,2023-01-27 12:22:43.000,1.129412,17.0
316325,3432056,10408954,eGFR(CKD-EPI) age >=18,68.69,2023-02-24 21:05:12.747,0.039200,25.0
316326,3432056,10573233,eGFR(CKD-EPI) age >=18,45.08,2023-04-10 09:51:32.587,-0.536591,44.0


# ✅ **check**


> **Check**
```
เช็คว่าแต่ละคนมีจำนวนสโลปที่คำนวณจากระยะเวลาห่างกันอย่างน้อย 3 เดือน
อย่างน้อย 2 สโลปไหม ถ้ามีไม่ครบให้ตัดคนนั้นทิ้ง
```


In [16]:
import pandas as pd

# Assuming you have a DataFrame 'eGFR_Patient' with index 'PatientUID' and columns 'datetime', 'slopes', and 'time_difference'

# Create an empty list to store the valid PatientUIDs
valid_patient_uids = []

# Group the DataFrame by index 'PatientUID'
grouped = eGFR_Patient.groupby(level='PatientUID')

# Iterate over each group
for patient_uid, group in grouped:
    # Check if the group has at least 2 rows
    if len(group) >= 2:
        # Calculate the minimum and maximum datetime in the group
        min_datetime = group['datetime'].min()
        max_datetime = group['datetime'].max()

        # Calculate the time difference in days between the minimum and maximum datetime
        time_diff = (max_datetime - min_datetime).days

        # Check if the time difference is at least 90 days (3 months)
        if time_diff >= 90:
            valid_patient_uids.append(patient_uid)

# Filter the original DataFrame based on the valid PatientUIDs
filtered_df = eGFR_Patient.loc[valid_patient_uids]

In [17]:
filtered_df

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2299       3640924          eGFR(CKD-EPI) age >=18  66.80   
           4020970          eGFR(CKD-EPI) age >=18  67.20   
           4286612          eGFR(CKD-EPI) age >=18  69.20   
           4546471          eGFR(CKD-EPI) age >=18  63.60   
           4801938          eGFR(CKD-EPI) age >=18  58.40   
...                                            ...    ...   
3417416    10528067         eGFR(CKD-EPI) age >=18  48.95   
3432056    10235244         eGFR(CKD-EPI) age >=18  48.51   
           10306813         eGFR(CKD-EPI) age >=18  67.71   
           10408954         eGFR(CKD-EPI) age >=18  68.69   
           10573233         eGFR(CKD-EPI) age >=18  45.08   

                                          datetime    slopes  time_difference  
PatientUID PatientVisitUID                                                     
2299       3640924         2016-11-11 09:40:06.053       NaN             87.0  
           4020970         2017-03-31 10:18:54.363  0.002857            140.0  
           4286612         2017-07-05 10:40:40.763  0.020833             96.0  
           4546471         2017-10-03 09:19:54.710 -0.062921             89.0  
           4801938         2018-01-04 09:03:28.847 -0.056522             92.0  
...                                            ...       ...              ...  
3417416    10528067        2023-03-28 13:59:40.430 -0.084091            110.0  
3432056    10235244        2023-01-09 12:31:53.220  0.168400             25.0  
           10306813        2023-01-27 12:22:43.000  1.129412             17.0  
           10408954        2023-02-24 21:05:12.747  0.039200             25.0  
           10573233        2023-04-10 09:51:32.587 -0.536591             44.0  

[313439 rows x 5 columns]

In [18]:
row_counts = filtered_df.groupby('PatientUID').size()


patient_uids_with_less_than_2_row = row_counts[row_counts < 2].index

patient_uids_with_2_row = row_counts[row_counts == 2].index

print("PatientUIDs with less than 2 row:") #dont have
print(patient_uids_with_less_than_2_row)

print("PatientUIDs with 2 row:")
print(patient_uids_with_2_row)

PatientUIDs with less than 2 row:
Int64Index([], dtype='int64', name='PatientUID')
PatientUIDs with 2 row:
Int64Index([   2378,    3370,    6798,    7849,    8857,    8881,    9107,
              10111,   11269,   11784,
            ...
            3385319, 3396335, 3397015, 3399742, 3405404, 3407146, 3410729,
            3412239, 3414078, 3417416],
           dtype='int64', name='PatientUID', length=1292)


In [19]:
test = filtered_df.reset_index() # check if time_diff > 90
test.head()

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
0,2299,3640924,eGFR(CKD-EPI) age >=18,66.8,2016-11-11 09:40:06.053,NaN,87.0
1,2299,4020970,eGFR(CKD-EPI) age >=18,67.2,2017-03-31 10:18:54.363,0.002857,140.0
2,2299,4286612,eGFR(CKD-EPI) age >=18,69.2,2017-07-05 10:40:40.763,0.020833,96.0
3,2299,4546471,eGFR(CKD-EPI) age >=18,63.6,2017-10-03 09:19:54.710,-0.062921,89.0
4,2299,4801938,eGFR(CKD-EPI) age >=18,58.4,2018-01-04 09:03:28.847,-0.056522,92.0


In [20]:
t[t['PatientUID']==3417416] # before cut

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
316314,3417416,10133709,eGFR(CKD-EPI) age >=18,58.20,2022-12-08 07:23:51.783,2.330667,15.0
316315,3417416,10528067,eGFR(CKD-EPI) age >=18,48.95,2023-03-28 13:59:40.430,-0.084091,110.0


In [21]:
test[test['PatientUID']==3417416] # check if time_diff > 90

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
313433,3417416,10133709,eGFR(CKD-EPI) age >=18,58.20,2022-12-08 07:23:51.783,2.330667,15.0
313434,3417416,10528067,eGFR(CKD-EPI) age >=18,48.95,2023-03-28 13:59:40.430,-0.084091,110.0


- **Delete each first rows slope of each PatientUID**

In [22]:
# Replace 'slopes' and 'time_difference' columns of the first row for each unique 'PatientUID' in DataFrame 'filtered_df'
for id in filtered_df.index.get_level_values('PatientUID').unique():
    first_row_index = filtered_df.loc[(id, slice(None)), :].index[0]
    filtered_df.loc[first_row_index, ('slopes', 'time_difference')] = None

# Print the updated DataFrame 'filtered_df'
filtered_df

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2299       3640924          eGFR(CKD-EPI) age >=18  66.80   
           4020970          eGFR(CKD-EPI) age >=18  67.20   
           4286612          eGFR(CKD-EPI) age >=18  69.20   
           4546471          eGFR(CKD-EPI) age >=18  63.60   
           4801938          eGFR(CKD-EPI) age >=18  58.40   
...                                            ...    ...   
3417416    10528067         eGFR(CKD-EPI) age >=18  48.95   
3432056    10235244         eGFR(CKD-EPI) age >=18  48.51   
           10306813         eGFR(CKD-EPI) age >=18  67.71   
           10408954         eGFR(CKD-EPI) age >=18  68.69   
           10573233         eGFR(CKD-EPI) age >=18  45.08   

                                          datetime    slopes  time_difference  
PatientUID PatientVisitUID                                                     
2299       3640924         2016-11-11 09:40:06.053       NaN              NaN  
           4020970         2017-03-31 10:18:54.363  0.002857            140.0  
           4286612         2017-07-05 10:40:40.763  0.020833             96.0  
           4546471         2017-10-03 09:19:54.710 -0.062921             89.0  
           4801938         2018-01-04 09:03:28.847 -0.056522             92.0  
...                                            ...       ...              ...  
3417416    10528067        2023-03-28 13:59:40.430 -0.084091            110.0  
3432056    10235244        2023-01-09 12:31:53.220       NaN              NaN  
           10306813        2023-01-27 12:22:43.000  1.129412             17.0  
           10408954        2023-02-24 21:05:12.747  0.039200             25.0  
           10573233        2023-04-10 09:51:32.587 -0.536591             44.0  

[313439 rows x 5 columns]

In [23]:
a = filtered_df.reset_index()

In [24]:
a

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
0,2299,3640924,eGFR(CKD-EPI) age >=18,66.80,2016-11-11 09:40:06.053,NaN,NaN
1,2299,4020970,eGFR(CKD-EPI) age >=18,67.20,2017-03-31 10:18:54.363,0.002857,140.0
2,2299,4286612,eGFR(CKD-EPI) age >=18,69.20,2017-07-05 10:40:40.763,0.020833,96.0
3,2299,4546471,eGFR(CKD-EPI) age >=18,63.60,2017-10-03 09:19:54.710,-0.062921,89.0
4,2299,4801938,eGFR(CKD-EPI) age >=18,58.40,2018-01-04 09:03:28.847,-0.056522,92.0
...,...,...,...,...,...,...,...
313434,3417416,10528067,eGFR(CKD-EPI) age >=18,48.95,2023-03-28 13:59:40.430,-0.084091,110.0
313435,3432056,10235244,eGFR(CKD-EPI) age >=18,48.51,2023-01-09 12:31:53.220,NaN,NaN
313436,3432056,10306813,eGFR(CKD-EPI) age >=18,67.71,2023-01-27 12:22:43.000,1.129412,17.0
313437,3432056,10408954,eGFR(CKD-EPI) age >=18,68.69,2023-02-24 21:05:12.747,0.039200,25.0


In [26]:
a['PatientUID'].nunique()

20368

In [25]:
a.to_excel('SlopeFiltered.xlsx', index=False)


**suggestion**
```
ควรจะเก็บแค่สโลปที่มีมากกว่า 3 rows เพราะถ้าเก็บสโลปอย่างน้อย 2 rows (2 slopes)
some patient will have only 1 slope cuz the fist slope must be deleted

```

